In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import warnings

from src.forecasting.Scalers import enrich_tensor

warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta

from model.patchtst  import PatchTST_SOTA
from model.quantile_head import QuantileRegressionHead
from utils.visualization import plot_predictions
from utils.data_loader import generate_data_from_df, invert_scale, invert_truth_scale, plot_unscaled_predictions

from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import LabelEncoder

from utils.labels import detect_head_and_shoulders, detect_double_bottom, label_patterns, create_windows
from utils.metrics import quantile_loss
from utils.classify_window import classify_window
# from utils.forecaster import forecaster

# from model.CNN.CNN1D import CNN1D_MultiChannel
# from model.CNN.CNN2D import CNN2D_MultiChannel

# from trainers.CNN1DTrainer import CNN1DTrainer, TimeSeries1DDataset
# from trainers.CNN2DTrainer import CNN2DTrainer, TimeSeries2DDataset

### Load data

In [2]:
filepath = "data"
filename = "NIFTY 50_minute_data.csv"
# filename = "NIFTY 50_daily_data.csv"

file = f"{filepath}/{filename}"

df_nifty50_full = pd.read_csv(file)
df_nifty50_full.shape

(932946, 6)

In [3]:
# extract all records since 01-01-2022

df_nifty50 = df_nifty50_full[df_nifty50_full['date'] > '2022-01-01 09:00:00']


df_nifty50.loc[:, "ema"] = ta.ema(df_nifty50['close'], length=15)
df_nifty50.loc[:, "rsi"] = ta.rsi(df_nifty50['close'], length=15)
df_nifty50.loc[:, "zscore"] = ta.zscore(df_nifty50['close'], length=15)

df_nifty50.dropna(inplace=True)
df_nifty50.reset_index(drop=True, inplace=True)
df_nifty50 = df_nifty50[['close','ema','rsi','zscore']]

/var/folders/cr/mwy_6lb106140xrnl93j65t80000gn/T/ipykernel_72490/3884146209.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nifty50.loc[:, "ema"] = ta.ema(df_nifty50['close'], length=15)
/var/folders/cr/mwy_6lb106140xrnl93j65t80000gn/T/ipykernel_72490/3884146209.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nifty50.loc[:, "rsi"] = ta.rsi(df_nifty50['close'], length=15)
/var/folders/cr/mwy_6lb106140xrnl93j65t80000gn/T/ipykernel_72490/3884146209.py:8: SettingWithCopyWarning: 
A value is trying

#### Note: Model performace improves when trained on < 100k records

### Transformer (PatchTST)

In [4]:
feature_cols = ['close','ema','rsi','zscore']
checkpoint_path = "models_results/checkpoints/model_guassian_en_manik.pt"

X, y, scaler = generate_data_from_df(df_nifty50, feature_cols, input_len=50, pred_len=10)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

seq_len, pred_len = 60, 10
quantiles = [0.1, 0.5, 0.9]

In [5]:

from src.forecasting.GaussianTransformer import CustomTransformerEncoderOnly


def load_model(checkpoint_path, input_len, pred_len, num_features):
    model = CustomTransformerEncoderOnly(d_model=4, nhead=2, num_encoder_layers=6, dim_feedforward=256)
    model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))
    model.eval()
    return model

def run_inference(model, test_loader, quantiles):
    model.eval()
    all_preds = []
    all_truth = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch)
            all_preds.append(preds)
            all_truth.append(y_batch)
    preds_tensor = torch.cat(all_preds, dim=0)
    truth_tensor = torch.cat(all_truth, dim=0)
    return preds_tensor, truth_tensor

### Market Simulation

In [6]:
from MarketSimulator import * 

In [7]:
WINDOW_SIZE = 10
PATTERN_LABELS = ['head_and_shoulders', 'double_bottom', 'ascending_triangle', 'none']

#### Data preparation 

In [8]:
def extract_windows(df, seq_len, step_size):
    windows = []
    for start in range(0, len(df) - seq_len + 1, step_size):
        end = start + seq_len
        window = df.iloc[start:end].copy()
        windows.append(window)
    return windows

In [9]:
num_days = 10
trading_hrs = 6.5
simulation_data_size = int(60 * trading_hrs * num_days) # minutes * hrs * days 
window_seq_length = 60
prediction_length = 10
step_size = WINDOW_SIZE
checkpoint_path = "models_results/checkpoints/model_guassian_en_manik.pt"

# Start at index 0 and then slide the window of size(WINDOW_SIZE) by step_size steps
simulation_windows = extract_windows(df_nifty50[-simulation_data_size:], window_seq_length, step_size)
simulation_windows

[           close           ema        rsi    zscore
 284066  22867.70  22866.889173  45.324360  0.848359
 284067  22875.55  22867.971777  50.053955  1.748719
 284068  22869.05  22868.106554  46.486459  0.685541
 284069  22865.50  22867.780735  44.625296  0.106473
 284070  22858.85  22866.664393  41.306130 -0.864049
 284071  22865.80  22866.556344  45.818697  0.115692
 284072  22871.65  22867.193051  49.331875  0.877095
 284073  22868.40  22867.343920  47.498622  0.423595
 284074  22872.45  22867.982180  49.980438  1.181347
 284075  22879.00  22869.359407  53.767446  1.873538
 284076  22873.50  22869.876981  50.338643  0.968105
 284077  22870.30  22869.929859  48.414024  0.333815
 284078  22868.10  22869.701126  47.087887 -0.161942
 284079  22879.35  22870.907236  53.992494  1.678620
 284080  22885.45  22872.725081  57.234529  2.140589
 284081  22890.75  22874.978196  59.867206  2.186653
 284082  22879.35  22875.524671  52.429001  0.744755
 284083  22883.60  22876.534088  54.678228  1.

#### Run forecasts

In [10]:
from src.forecasting.StockSeriesForcasterEncoder import StockSeriesForecasterEncoder
checkpoint_path = "models_results/checkpoints/model_guassian_en_manik.pt"
checkpoint_path_ed = "models_results/checkpoints/model_guassian_ed_manik.pt"
model = CustomTransformerEncoderOnly(d_model=4, nhead=2, num_encoder_layers=6, dim_feedforward=256)
model.load_state_dict(torch.load(checkpoint_path, weights_only=True))
model.eval()
criterion = nn.MSELoss()

forecaster = StockSeriesForecasterEncoder(model, None, criterion)

In [11]:
def reshape_forecast_for_cnn(forecast_tensor):
    """
    Input: forecast_tensor of shape (1, T, Q) or (T, Q=3)
    Output: reshaped tensor of shape (T, 4) for CNN
    """
    if isinstance(forecast_tensor, np.ndarray):
        forecast_np = forecast_tensor
    else:
        forecast_np = forecast_tensor.detach().cpu().numpy()

    if forecast_np.ndim == 3:
        forecast_np = forecast_np.squeeze(0)  # (T, Q)

    # Add 4th channel (mean of quantiles)
    #avg_col = np.mean(forecast_np, axis=-1, keepdims=True)
    #cnn_input = np.concatenate([forecast_np, avg_col], axis=-1)  # (T, 4)
    #print(cnn_input)
    return torch.tensor(forecast_np, dtype=torch.float32)  # (T, 4)

In [12]:
# Prepare the simulation data
# This is a list of pairs of input sequence (60 timesteps) - that goes as input to the transoformer
# and the classification label for the input sequence

simulation_data = []

for fcast in simulation_windows:

    classified_forecast = dict()
    
    #forecast = run_model_prediction(model, scaler, fcast,feature_cols, window_seq_length)
    fcast_tensor = torch.tensor(fcast['close'].values, dtype=torch.float32)
    fcast_tensor = fcast_tensor.unsqueeze(0).unsqueeze(2)

    forecast = forecaster.autoregressive_predict(fcast_tensor, prediction_length)
    classified_forecast['window'] = fcast['close']
    classified_forecast['forecast'] = forecast.unsqueeze(2)
    classified_forecast['label'] = classify_window(reshape_forecast_for_cnn(enrich_tensor(forecast.unsqueeze(2))), model_type='2d', model_path="models_results/checkpoints/CNN2D_MULTICHANNEL.pt", PATTERN_LABELS=PATTERN_LABELS)
    simulation_data.append(classified_forecast)
    

In [13]:
len(simulation_data)

385

In [14]:
trader = Trader()

# have to use a very high price tolerance as the forecast is way off the actual close price
simulator = MarketSimulator(processed_data=simulation_data, initial_budget=100000, price_tolerance=5000)

trade_log = simulator.simulate()

In [15]:
trade_log

[(6, 'buy', 22795.171875, 77204.828125),
 (6, 'sell', 22824.291015625, 100029.119140625),
 (15, 'buy', 22868.005859375, 77161.11328125),
 (15, 'sell', 22887.0390625, 100048.15234375),
 (24, 'buy', 23021.162109375, 77026.990234375),
 (24, 'sell', 23022.623046875, 100049.61328125),
 (31, 'buy', 23021.880859375, 77027.732421875),
 (31, 'sell', 23022.140625, 100049.873046875),
 (46, 'buy', 22880.228515625, 77169.64453125),
 (46, 'sell', 22968.14453125, 100137.7890625),
 (57, 'buy', 23081.71875, 77056.0703125),
 (57, 'sell', 23081.71875, 100137.7890625),
 (73, 'buy', 23163.6015625, 76974.1875),
 (73, 'sell', 23163.6015625, 100137.7890625),
 (82, 'buy', 23146.015625, 76991.7734375),
 (82, 'sell', 23146.94140625, 100138.71484375),
 (85, 'buy', 23150.892578125, 76987.822265625),
 (85, 'sell', 23150.892578125, 100138.71484375),
 (97, 'buy', 23258.109375, 76880.60546875),
 (97, 'sell', 23269.568359375, 100150.173828125),
 (142, 'buy', 23462.369140625, 76687.8046875),
 (142, 'sell', 23463.1777343